In [ ]:
import os 
import sys
from tqdm import tqdm
import importlib
import numpy as np
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt

module_path = '/home/lun/project-basileus/seq-gan/'
if module_path not in sys.path:
    sys.path.append(module_path)
    
from tensorflow_probability import distributions as tfd    

In [ ]:
if 'sgtlstm' in sys.modules:
    importlib.reload(sys.modules['sgtlstm'])
    
from sgtlstm.utils import create_dataset
from sgtlstm.SeqGan import build_G, build_D
from sgtlstm.pretrain import pretrain_generator, pretrain_discriminator, create_self_regression_data_batch
from sgtlstm.TimeLSTM import TimeLSTM0, TimeLSTM1, TimeLSTM2, TimeLSTM3

import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD

## Load data 

In [ ]:
pos_data_path = '/home/lun/project-basileus/seq-gan/data/long_seqs_v8/positive_long_sequences.pickle'
neg_data_path = '/home/lun/project-basileus/seq-gan/data/long_seqs_v8/negative_long_sequences.pickle'
all_data_path = '/home/lun/project-basileus/seq-gan/data/long_seqs_v8/all_long_sequences.pickle'

def load_sequence_from_pickle_to_numpy(pickle_file_path):
    """
        A list of sequence in format of (event_type, time_delta)
    :param pickle_file_path: e.g. /.../project-basileus/seq-gan/data/fixed_length/valid_sequences.pickle
    :return: (event_type_seqs, time_delta)
    """
    with open(pickle_file_path, 'rb') as f:
        raw_seqs = pickle.load(f)

    if not raw_seqs or not raw_seqs[0]:
        return np.array([]), np.array([])

    N = len(raw_seqs)
    T = len(raw_seqs[0])
    
    seqs = np.array(raw_seqs)
#     print(seqs.shape)
    
    et_seqs = seqs[:, :, 0].astype(np.float64).reshape((N, T, 1))
    ts_seqs = seqs[:, :, 1].astype(np.float64).reshape((N, T, 1))
    return et_seqs, ts_seqs
    
pos_event_type_seqs, pos_timestamp_seqs = load_sequence_from_pickle_to_numpy(pos_data_path)
neg_event_type_seqs, neg_timestamp_seqs = load_sequence_from_pickle_to_numpy(neg_data_path)
# all_event_type_seqs, all_timestamp_seqs = load_sequence_from_pickle_to_numpy(all_data_path)
all_event_type_seqs = np.concatenate([pos_event_type_seqs, neg_event_type_seqs], axis=0)
all_timestamp_seqs = np.concatenate([pos_timestamp_seqs, neg_timestamp_seqs], axis=0)

In [ ]:
pos_event_type_seqs.shape[0] / neg_event_type_seqs.shape[0]

In [ ]:
all_timestamp_seqs.shape

## Global Variables 

In [ ]:
BATCH_SIZE = 16
T = 20 + 1
VOCAB = ['END/PADDING', 'INIT', 'start', 'view', 'click', 'install']
EVENT_VOCAB_DIM = len(VOCAB)
EMB_DIM = 6
HIDDEN_DIM = 100

END_TOKEN = 0
MAX_TIME = 1024

## G and D

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Input, LSTM, Embedding, Reshape, Dense, Dropout
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
from tensorflow.keras import regularizers

from sgtlstm.TimeLSTM import TimeLSTM0, TimeLSTM1, TimeLSTM2, TimeLSTM3

tf.keras.backend.set_floatx('float64')

def build_D_2(T, event_vocab_dim, emb_dim, hidden_dim=11):
    """
        Build a discriminator for event type sequence of shape (batch_size, T, input_dim)
        and input event type sequence of shape (batch_size, T, 1)
    :param T: length of the sequence
    :param event_vocab_dim: size of event vocabulary ['na', 'init', 'start', 'view', 'click', 'install']
    :param emb_dim: dimension of the embedding layer output for event type
    :param hidden_dim: dimension hidden of the time lstm cell
    :return: discriminator D
    """
    # Time-LSTM:
    i_et = Input(shape=(T, 1), name='event_type')  # input of discrete feature event type
    i_ts = Input(shape=(T, 1), name='time_delta')  # input of continuous feature timestamp
    mask_layer = tf.keras.layers.Masking(mask_value=0., input_shape=(T, 1))
    masked_ts = mask_layer(i_ts)
    masked_et = mask_layer(i_et)

    embed0 = Embedding(input_dim=event_vocab_dim, output_dim=emb_dim, input_length=T, mask_zero=True)(masked_et)
    embed0 = Reshape((T, emb_dim))(embed0)  # shape=[Batch_size, T, emb_dim]
    merged0 = tf.keras.layers.concatenate([embed0, masked_ts], axis=2)  # # shape=[Batch_size, T, emb_dim + time_dim]

    hm, tm = TimeLSTM1(hidden_dim, activation='selu', name='time_lstm', return_sequences=False)(merged0)

    time_comb = tf.keras.layers.concatenate([hm, tm], axis=1)

    # predicted real prob
    real_prob = Dense(1, activation='sigmoid', name='fraud_prob', kernel_regularizer=regularizers.l1_l2(l1=1e-3, l2=1e-3))(
        time_comb)

    discriminator = Model(
        inputs=[i_et, i_ts],
        outputs=[real_prob])

    return discriminator


def build_G_2(batch_size, event_vocab_dim, emb_dim, hidden_dim=11):
    """
        Build a generator for event type sequence of shape (batch_size, T, input_dim)
        and input event type sequence of shape (batch_size, T, 1)
    :param batch_size: batch size must been specified at generator
    :param event_vocab_dim: size of event vocabulary ['na', 'start', 'click', 'install']
    :param emb_dim: dimension of the embedding layer output for event type
    :param hidden_dim: dimension hidden of the time lstm cell
    :return:
    """
    # Time-LSTM:
    i_et = Input(batch_shape=(batch_size, None, 1), name='event_type')  # input of discrete feature event type
    i_ts = Input(batch_shape=(batch_size, None, 1), name='time_delta')  # input of continuous feature timestamp

    mask_layer = tf.keras.layers.Masking(mask_value=0., input_shape=(None, 1))
    masked_et = mask_layer(i_et)
    masked_ts = mask_layer(i_ts)

    embed0 = Embedding(input_dim=event_vocab_dim, output_dim=emb_dim, mask_zero=True)(masked_et)
    embed0 = Reshape([1, emb_dim])(embed0)
    merged0 = tf.keras.layers.concatenate([embed0, masked_ts], axis=2)

    hm, tm = TimeLSTM1(hidden_dim, activation='selu', name='time_lstm',
                       stateful=True, return_sequences=False)(merged0)
    
    time_comb = tf.keras.layers.concatenate([hm, tm], axis=1)
    
    dense_time = Dense(hidden_dim // 2, activation='linear', name='dense_time')(time_comb)
    dense_token = Dropout(rate=0.4)(dense_time)
    time_out = Dense(1 + 1, activation='linear', name='output')(dense_token)
    time_out = tfp.layers.DistributionLambda(
        lambda t: tfd.Normal(loc=t[..., :1],
                             scale=1 + tf.math.softplus(t[..., 1:])),
        name='Normal')(time_out)

    # predicted prob of next token
    dense_token = Dense(hidden_dim // 2, activation='linear', name='dense_token')(time_comb)
    dense_token = Dropout(rate=0.4)(dense_token)
    token_prob = Dense(event_vocab_dim, activation='softmax', name='token_prob')(dense_token)
    generator = Model(
        inputs=[i_et, i_ts],
        outputs=[token_prob, time_out])
    return generator


In [ ]:
# generator = build_G(
#     batch_size=BATCH_SIZE,
#     event_vocab_dim = EVENT_VOCAB_DIM,
#     emb_dim = EMB_DIM,
#     hidden_dim= HIDDEN_DIM)

# discriminator = build_D(
#     T = T,
#     event_vocab_dim = EVENT_VOCAB_DIM,
#     emb_dim = EMB_DIM,
#     hidden_dim= HIDDEN_DIM,
# )

## Pretrain G

In [ ]:
pretrain_G_et = all_event_type_seqs
pretrain_G_ts = all_timestamp_seqs
pretrain_G_labels = np.ones((all_event_type_seqs.shape[0], 1))

pretrain_G_features = (pretrain_G_et, pretrain_G_ts)
N_pretrain_G = pretrain_G_et.shape[0]
N_pretrain_G

In [ ]:
EPOCHS = 1
_TOTAL_STEPS = int(EPOCHS * N_pretrain_G / BATCH_SIZE)


pretrain_G_dataset = create_dataset(pretrain_G_features,
                                  pretrain_G_labels,
                                  batch_size=BATCH_SIZE,
                                  epochs=EPOCHS,
                                  buffer_size=N_pretrain_G)

pretrain_gen_ce_loss_history = []
pretrain_gen_gaussian_loss_history = []

pretrained_generator = build_G_2(
    batch_size=BATCH_SIZE,
    event_vocab_dim = EVENT_VOCAB_DIM,
    emb_dim = EMB_DIM,
    hidden_dim= HIDDEN_DIM)

In [ ]:
_TOTAL_STEPS

In [ ]:
step = 0
OPTIMIZER = Adam(lr=1e-4)
WEIGHT_GAUSSIAN_LOSS = 1

for feature_sample, _ in tqdm(pretrain_G_dataset.take(_TOTAL_STEPS)):
    step += 1
    print('Training Step:', step)
        
    gen_ce_loss, gen_gaussian_loss =  pretrain_generator(feature_sample, 
                                                         pretrained_generator,
                                                         verbose=True, 
                                                         weight_gaussian_loss=WEIGHT_GAUSSIAN_LOSS, 
                                                         optimizer=OPTIMIZER)
        
    pretrain_gen_ce_loss_history.append(gen_ce_loss.numpy())
    pretrain_gen_gaussian_loss_history.append(gen_gaussian_loss.numpy())

In [ ]:
x = range(len(pretrain_gen_ce_loss_history))
plt.figure(dpi=100)
plt.plot(x, pretrain_gen_ce_loss_history)
plt.title('Pre-training Generator Categorical Cross-Entropy Loss History')
plt.xlabel('Pre-training steps')

x = range(len(pretrain_gen_gaussian_loss_history))
plt.figure(dpi=100)
plt.plot(x, pretrain_gen_gaussian_loss_history)
plt.title('Pre-training Generator Gaussian Loss History')
plt.xlabel('training steps')

In [ ]:
loss_save_dir = '/home/lun/project-basileus/seq-gan/experiment_results/long_seqs_v7/init_pretrained/loss'
if not os.path.exists(loss_save_dir):
    os.makedirs(loss_save_dir)
    
with open(os.path.join(loss_save_dir, 'pretrain_gen_ce_loss_history.pickle'), 'wb') as f:
    pickle.dump(pretrain_gen_ce_loss_history, f)

with open(os.path.join(loss_save_dir, 'pretrain_gen_gaussian_loss_history.pickle'), 'wb') as f:
    pickle.dump(pretrain_gen_gaussian_loss_history, f)

### Save Pretrained G

In [ ]:
if not os.path.exists('/home/lun/project-basileus/seq-gan/experiment_results/long_seqs_v7/init_pretrained/pretrained_gen_weights'):
    os.makedirs('./experiment_results/long_seqs_v7/init_pretrained/pretrained_gen_weights')
    
G_save_path = '/home/lun/project-basileus/seq-gan/experiment_results/long_seqs_v7/init_pretrained/pretrained_gen_weights/model_dropout.tf'
pretrained_generator.save_weights(G_save_path)

In [ ]:
G_save_path = '/home/lun/project-basileus/seq-gan/experiment_results/long_seqs_v7/init_pretrained/pretrained_gen_weights/model_dropout.tf'

reload_pretrained_gen = build_G_2(
    batch_size = BATCH_SIZE,
    event_vocab_dim = EVENT_VOCAB_DIM,
    emb_dim = EMB_DIM,
    hidden_dim= HIDDEN_DIM,
)

reload_pretrained_gen.build(input_shape=((BATCH_SIZE, T, 1), (BATCH_SIZE, T, 1)))
reload_pretrained_gen.load_weights(G_save_path)

In [ ]:
reload_pretrained_gen.summary()

## Pretrain D 

In [ ]:
pretrain_D_et = np.concatenate([pos_event_type_seqs, neg_event_type_seqs], axis=0)
pretrain_D_ts = np.concatenate([pos_timestamp_seqs, neg_timestamp_seqs], axis=0)

pretrain_D_labels = np.concatenate([np.ones((pos_event_type_seqs.shape[0], 1)), 
                                  np.zeros((neg_event_type_seqs.shape[0], 1))
                                 ], axis=0)
pretrain_D_features = (pretrain_D_et, pretrain_D_ts)
N_pretrain_D = pretrain_D_ts.shape[0]

In [ ]:
WEIGHT_GAUSSIAN_LOSS = 1e-2
OPTIMIZER = Adam(lr=1e-3)

EPOCHS = 1
_TOTAL_STEPS = int(EPOCHS * N_pretrain_D / BATCH_SIZE)
# _TOTAL_STEPS = 1000

pretrain_disc_token_loss_history = []
pretrain_disc_gaussian_loss_history = []


pretrain_D_dataset = create_dataset(pretrain_D_features,
                                  pretrain_D_labels,
                                  batch_size=BATCH_SIZE,
                                  epochs=EPOCHS,
                                  buffer_size=N_pretrain_D)

pretrained_discriminator = build_D_2(
    T = T,
    event_vocab_dim = EVENT_VOCAB_DIM,
    emb_dim = EMB_DIM,
    hidden_dim= HIDDEN_DIM,
)

In [ ]:
_TOTAL_STEPS

In [ ]:
step = 0

for features_batch, real_labels in tqdm(pretrain_D_dataset.take(_TOTAL_STEPS)):
    step += 1
    print('Training Step:', step)
        
    disc_token_loss = pretrain_discriminator(features_batch, real_labels, pretrained_discriminator, verbose=True, 
                                                                 weight_gaussian_loss=WEIGHT_GAUSSIAN_LOSS, optimizer=OPTIMIZER)
    pretrain_disc_token_loss_history.append(disc_token_loss.numpy())

### pretrain D: Loss over training

In [ ]:
x = range(len(pretrain_disc_token_loss_history))
plt.figure(dpi=100)
plt.plot(x, pretrain_disc_token_loss_history)
plt.title('Pre-training Discriminator CE Loss History')
plt.xlabel('Pre-training steps')

In [ ]:
loss_save_dir = '/home/lun/project-basileus/seq-gan/experiment_results/long_seqs_v7/init_pretrained/loss'
if not os.path.exists(loss_save_dir):
    os.makedirs(loss_save_dir)
    
with open(os.path.join(loss_save_dir, 'pretrain_disc_token_loss_history.pickle'), 'wb') as f:
    pickle.dump(pretrain_disc_token_loss_history, f)

### Save Pretrained D

In [ ]:
if not os.path.exists('/home/lun/project-basileus/seq-gan/experiment_results/long_seqs_v7/init_pretrained/pretrained_disc_weights'):
    os.makedirs('/home/lun/project-basileus/seq-gan/experiment_results/long_seqs_v7/init_pretrained/pretrained_disc_weights')
    
D_save_path = '/home/lun/project-basileus/seq-gan/experiment_results/long_seqs_v7/init_pretrained/pretrained_disc_weights/model.tf'
pretrained_discriminator.save_weights(D_save_path)

In [ ]:
reload_pretrained_disc = build_D(
    T = T,
    event_vocab_dim = EVENT_VOCAB_DIM,
    emb_dim = EMB_DIM,
    hidden_dim= HIDDEN_DIM,
)

reload_pretrained_disc.build(input_shape=((BATCH_SIZE, T, 1), (BATCH_SIZE, T, 1)))
reload_pretrained_disc.load_weights(D_save_path)

In [ ]:
reload_pretrained_disc.summary()

## Generate and predict seqs

In [ ]:
def generate_batch_sequence_by_rollout(
        G, batch_size, T, end_token=0, init_token=1.0, max_time=1024, verbose=False):
    # Begin from dummy init state (init_token=1, init_timestamp=0.0)
    curr_state_et = tf.ones([batch_size, 1, 1], dtype=tf.float64)
    curr_state_ts = tf.zeros([batch_size, 1, 1], dtype=tf.float64)

    all_state_et = curr_state_et
    all_state_ts = curr_state_ts

    episode_token_probs = tf.constant(1., dtype=tf.float64, shape=(batch_size, 1))
    gaussian_log = tf.constant(0., dtype=tf.float64, shape=(batch_size, 1))

    G.reset_states()

    for step in range(1, T):  # sequence length
        token_prob, time_out = G([curr_state_et, curr_state_ts])

        sampled_et = tf.random.categorical(tf.math.log(token_prob), num_samples=1, dtype=tf.int32)
        sampled_et = tf.reshape(sampled_et, [batch_size, 1, 1])

        # get the chosen token probability per batch for each step
        batch_sample_et = tf.reshape(sampled_et, (batch_size, 1))
        batch_ind = tf.reshape(tf.range(0, batch_size), (batch_size, 1))
        batch_sample_et_2d = tf.concat([batch_ind, batch_sample_et], axis=1)

        sampled_token_prob = tf.reshape(tf.gather_nd(token_prob, batch_sample_et_2d), (batch_size, 1))
        episode_token_probs = tf.concat([episode_token_probs, sampled_token_prob], axis=1)

        # cast sampled_et into float
        sampled_et = tf.cast(sampled_et, dtype=tf.float64)

        # stop genererating once hit end_token
        cond_end_token = tf.equal(curr_state_et, end_token)
        curr_state_et = tf.where(cond_end_token, curr_state_et, sampled_et)
        all_state_et = tf.concat([all_state_et, curr_state_et], axis=1)

        # generate one timstamp using time_out
        sampled_ts_raw = time_out.sample()
        sampled_ts = tf.clip_by_value(tf.reshape(sampled_ts_raw, (batch_size, 1, 1))
                                      , clip_value_min=1, clip_value_max=max_time)

        # get the gaussian log likelihood for the sampled timestamps
        sampled_gaussian_log = time_out.log_prob(sampled_ts_raw)
        gaussian_log = tf.concat([gaussian_log, sampled_gaussian_log], axis=1)

        # stop generating once hit end_token
        curr_state_ts = tf.where(cond_end_token, curr_state_et, sampled_ts)
        all_state_ts = tf.concat([all_state_ts, curr_state_ts], axis=1)

    return all_state_et, all_state_ts, episode_token_probs, gaussian_log


In [ ]:
def generate_sequences(N_gen, generator, batch_size, T, recover_to_timestamp=True):
    """
        Generate sequences batch per batch
    :param N_gen: total number of seqs to be generated
    :param generator:
    :param batch_size:
    :param T:
    :param recover_to_timestamp: whether to recover time deltas to absolute timestamps
    :return: a python list of shape [N_gen, T, 2]
    """
    all_type_seq = None
    all_time_seq = None
    N = 0

    while N < N_gen:
        batch_state_et, batch_state_ts, _, _ = generate_batch_sequence_by_rollout(generator, batch_size, T,
                                                                                  end_token=0, init_token=1.0,
                                                                                  max_time=1024, verbose=False)

        batch_type_seq = batch_state_et.numpy()
        batch_time_seq = batch_state_ts.numpy()

        # recover time delta to time stamps
        if recover_to_timestamp:
            batch_time_seq = np.cumsum(batch_time_seq, axis=1)

        if all_type_seq is None:
            all_type_seq = batch_type_seq
        else:
            all_type_seq = np.concatenate([all_type_seq, batch_type_seq], axis=0)

        if all_time_seq is None:
            all_time_seq = batch_time_seq
        else:
            all_time_seq = np.concatenate([all_time_seq, batch_time_seq], axis=0)

        N += batch_size

    # concat type and time in depth
    concated_seq_list = np.concatenate([all_type_seq, all_time_seq], axis=2).tolist()

    return concated_seq_list[:N_gen]


In [ ]:
N_gen = 100
generator = pretrained_generator

generated_seqs = generate_sequences(N_gen, generator, batch_size=BATCH_SIZE, T=T, recover_to_timestamp=False)

In [ ]:
generated_seqs = np.array(generated_seqs)
generated_seqs.shape

In [ ]:
generated_seqs = np.array(generated_seqs)
pred_generated = reload_pretrained_disc((generated_seqs[:,:,[0]], generated_seqs[:,:,[1]]))
(pred_generated > 0.9).numpy().sum()

In [ ]:
pos_event_type_seqs.shape

In [ ]:
pos_event_type_seqs.mean(axis=0)

In [ ]:
neg_event_type_seqs.mean(axis=0)

In [ ]:
pos_event_type_seqs.mean(axis=0) - neg_event_type_seqs.mean(axis=0)